In [1]:
import simpy
import numpy as np

In [2]:
def customer(env, mu, counter, waiting):
    # Time of customer arrival
    arrival = env.now

    # Time for service
    time = np.random.exponential(1 / mu)
    
    # Wait until customer can get serviced
    with counter.request(priority=time) as req:
        yield req
        
        # Add waiting time
        waiting.append(env.now - arrival)
        # Update time
        yield env.timeout(time)
    

In [3]:
def source(env, number, arrival, capacity, counter, waiting):
    # Creating customers
    for i in range(number):
        # Create the customer
        c = customer(env, capacity, counter, waiting)
        env.process(c)

        # Time for new person to arrive
        t = np.random.exponential(1 / arrival)
        
        # Update time
        yield env.timeout(t)  

In [6]:
# Parameters
customers = 100000
mu = 1
rho = 0.9
n = 1
arrival = rho * n * mu
print(f'lambda = {arrival}')
print(f'mu = {mu}')

# Array for waiting time
waiting = []

# Create environment
env = simpy.Environment()
np.random.seed(42)

# Create process
counter = simpy.PriorityResource(env, capacity=n)
env.process(source(env, customers, arrival, rho, counter, waiting))
env.run()


lambda = 0.9
mu = 1


In [8]:
# Get mean and standard deviation
waiting_mean = np.mean(waiting)
waiting_std = np.std(waiting, ddof=1)

# Get confidence interval
waiting_conf = 1.96 * waiting_std / np.sqrt(len(waiting))

# Print result?
print(f'{waiting_mean} +- {waiting_conf}')


38.876016998782184 +- 5.41339202480721
